In [9]:
%load_ext autoreload
%autoreload 2
%load_ext dotenv
%dotenv

import sys
import logging
import ipytest
import warnings
from pathlib import Path


CODE_FOLDER = Path("code")
CODE_FOLDER.mkdir(parents=True, exist_ok=True)
INFERENCE_CODE_FOLDER = CODE_FOLDER / "inference"
INFERENCE_CODE_FOLDER.mkdir(parents=True, exist_ok=True)

sys.path.extend([f"./{CODE_FOLDER}", f"./{INFERENCE_CODE_FOLDER}"])

ipytest.autoconfig(raise_on_error=True)

warnings.filterwarnings('ignore')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
The dotenv extension is already loaded. To reload it, use:
  %reload_ext dotenv


In [10]:
# !aws s3api create-bucket --bucket football-data-kamil --create-bucket-configuration LocationConstraint=eu-north-1

In [11]:
import sagemaker
import boto3

sagemaker_session = sagemaker.session.Session()
sagemaker_client = boto3.client("sagemaker")
iam_client = boto3.client("iam")
region = boto3.Session().region_name

In [12]:
import os
from sagemaker.workflow.pipeline_context import PipelineSession, LocalPipelineSession

DUMMY_ROLE = "arn:aws:iam::111111111111:role/service-role/AmazonSageMaker-ExecutionRole-11111111111111"
BUCKET = os.environ["BUCKET"]
S3_LOCATION = f"s3://{BUCKET}/football"
LOCAL_MODE = False

role = os.environ["ROLE"]

architecture = !(uname -m)
IS_ARM64_ARCHITECTURE = architecture[0] == "arm64"

if LOCAL_MODE:
    config = {
        "session": LocalPipelineSession(default_bucket=BUCKET),
        "instance_type": "local",
        "image": "sagemaker-xgboost-training-toolkit-local" if IS_ARM64_ARCHITECTURE else None
    }
else:
    config = {
        "session": PipelineSession(default_bucket=BUCKET) if not LOCAL_MODE else None,
        "instance_type": "ml.c5.xlarge",
        "image": None,
    }

config["framework_version"] = "1.7-1"
config["py_version"] = "py310"

In [13]:
from sagemaker.s3 import S3Uploader

df_local_path = str(os.environ['DATA_FILEPATH_X'])
y_local_path = str(os.environ['DATA_FILEPATH_Y'])

# S3Uploader.upload(local_path=df_local_path, desired_s3_uri=f"{S3_LOCATION}/data", sagemaker_session=config['session'])
# S3Uploader.upload(local_path=y_local_path, desired_s3_uri=f"{S3_LOCATION}/data", sagemaker_session=config['session'])

In [14]:
from sagemaker.workflow.steps import CacheConfig

cache_config = CacheConfig(enable_caching=True, expire_after="15d")

In [15]:
from sagemaker.workflow.pipeline_definition_config import PipelineDefinitionConfig
from sagemaker.sklearn import SKLearnProcessor
from sagemaker.workflow.steps import ProcessingStep
from sagemaker.processing import ProcessingInput, ProcessingOutput
from sagemaker.workflow.parameters import ParameterString
from sagemaker.workflow.retry import (
    SageMakerJobExceptionTypeEnum,
)

retry_policy = {
    "ExceptionType": SageMakerJobExceptionTypeEnum.INTERNAL_ERROR,
    "IntervalSeconds": 2,
    "BackoffRate": 2,
    "MaxAttempts": 5,
    "ExpireAfterMin": 1
}

pipeline_definition_config = PipelineDefinitionConfig(use_custom_job_prefix=True)

dataset_location = ParameterString(
    name="dataset_location",
    default_value=f"{S3_LOCATION}/data",
)

processor = SKLearnProcessor(
    base_job_name="split-and-transform-data",
    framework_version="1.2-1",
    instance_type=config["instance_type"],
    instance_count=1,
    role=role,
    sagemaker_session=config['session'],
)

split_and_transform_data_step = ProcessingStep(
    name="split-and-transform-data",
    step_args=processor.run(
        code=f"{CODE_FOLDER}/preprocessor.py",  # Make sure CODE_FOLDER is a string or handled if ParameterString
        inputs=[
            ProcessingInput(source=dataset_location, destination="/opt/ml/processing/input"),  # Convert to string if needed
        ],
        outputs=[
            ProcessingOutput(output_name="train", source="/opt/ml/processing/train"),
            ProcessingOutput(output_name="validation", source="/opt/ml/processing/validation"),
            ProcessingOutput(output_name="test", source="/opt/ml/processing/test"),
            ProcessingOutput(output_name="model", source="/opt/ml/processing/model"),
            ProcessingOutput(output_name="train-baseline", source="/opt/ml/processing/train-baseline"),
            ProcessingOutput(output_name="test-baseline", source="/opt/ml/processing/test-baseline"),
        ]
    ),
    cache_config=cache_config,
    retry_policies=[
        retry_policy
    ]
)



In [8]:
from sagemaker.workflow.pipeline import Pipeline

# session1_pipeline = Pipeline(
#     name="session1-pipeline",
#     parameters=[dataset_location],
#     steps=[
#         split_and_transform_data_step,
#     ],
#     pipeline_definition_config=pipeline_definition_config,
#     sagemaker_session=config['session'],
# )
# 
# session1_pipeline.upsert(role_arn=config["role"])
# session1_pipeline.start()

In [9]:
from sagemaker.xgboost import XGBoost

use_spot_instances = True and not LOCAL_MODE
max_run = 1000
max_wait = 1200 if use_spot_instances else None
instance_type = "ml.m5.2xlarge"

xgboost = XGBoost(
    base_job_name="training",
    entry_point=f"{CODE_FOLDER}/train.py",
    role=role,
    instance_count=1,
    instance_type=config['instance_type'],
    framework_version=config['framework_version'],
    disable_profiler=True,
    use_spot_instances=use_spot_instances,
    max_run=max_run,
    max_wait=max_wait,
    sagemaker_session=config['session']
)

INFO:sagemaker.image_uris:Ignoring unnecessary Python version: py3.
INFO:sagemaker.image_uris:Ignoring unnecessary instance type: ml.c5.xlarge.


In [10]:
from sagemaker.workflow.steps import TrainingStep
from sagemaker.inputs import TrainingInput


def create_training_step(estimator):
    """Create a SageMaker TrainingStep using the provided estimator."""
    return TrainingStep(
        name="train-model",
        step_args=estimator.fit(
            inputs={
                "train": TrainingInput(
                    s3_data=split_and_transform_data_step.properties.ProcessingOutputConfig.Outputs[
                        "train"
                    ].S3Output.S3Uri,
                    content_type="text/csv",
                ),
                "validation": TrainingInput(
                    s3_data=split_and_transform_data_step.properties.ProcessingOutputConfig.Outputs[
                        "validation"
                    ].S3Output.S3Uri,
                    content_type="text/csv",
                ),
            },
        ),
        cache_config=cache_config,
    )

train_model_step = create_training_step(xgboost)

In [11]:
USE_TUNING_STEP = False and not LOCAL_MODE

In [12]:
from sagemaker.tuner import HyperparameterTuner
from sagemaker.parameter import IntegerParameter, ContinuousParameter

hyperparameter_ranges = {
    'eta': ContinuousParameter(min_value=0.05, max_value=0.3, scaling_type="Logarithmic"),
    'max_depth': IntegerParameter(min_value=5, max_value=15, scaling_type="Auto"),
    'subsample': ContinuousParameter(min_value=0.7, max_value=1.0, scaling_type="Auto"),
    'colsample_bytree': ContinuousParameter(min_value=0.7, max_value=1.0, scaling_type="Logarithmic"),
    'lambda': ContinuousParameter(min_value=5, max_value=12, scaling_type="Logarithmic"),
    'alpha': ContinuousParameter(min_value=1, max_value=10, scaling_type="Logarithmic"),
    'min_child_weight': ContinuousParameter(min_value=0.4, max_value=1.0, scaling_type="Auto"),
}

objective_type = "Maximize"
metric_definitions = [
    {'Name': 'validation:logloss',
     'Regex': r".*\[[0-9]+\].*#011validation_0-logloss:([-+]?[0-9]*\.?[0-9]+(?:[eE][-+]?[0-9]+)?).*"
     },
    {'Name': 'validation:f1',
     'Regex': 'F1 score: ([0-9\\.]+)'
     },
]
metric_name = "validation:f1"
strategy = "Bayesian"

tuner = HyperparameterTuner(
    estimator=xgboost,
    objective_metric_name=metric_name,
    objective_type=objective_type,
    hyperparameter_ranges=hyperparameter_ranges,
    metric_definitions=metric_definitions,
    max_jobs=8,
    max_parallel_jobs=2,
    early_stopping_type='Auto',
)

In [13]:
from sagemaker.workflow.steps import TuningStep

tune_model_step = TuningStep(
    name="tune-model",
    step_args=tuner.fit(
        inputs={
            "train": TrainingInput(
                s3_data=split_and_transform_data_step.properties.ProcessingOutputConfig.Outputs[
                    "train"
                ].S3Output.S3Uri,
                content_type="text/csv"
            ),
            "validation": TrainingInput(
                s3_data=split_and_transform_data_step.properties.ProcessingOutputConfig.Outputs[
                    "validation"
                ].S3Output.S3Uri,
                content_type="text/csv"
            )
        },
    ),
    cache_config=cache_config
)

In [14]:
# session2_pipeline = Pipeline(
#     name="session2-pipeline",
#     parameters=[dataset_location],
#     steps=[
#         split_and_transform_data_step,
#         tune_model_step if USE_TUNING_STEP else train_model_step,
#     ],
#     pipeline_definition_config=pipeline_definition_config,
#     sagemaker_session=config["session"],
# )
# 
# session2_pipeline.upsert(role_arn=config['role'])
# session2_pipeline.start()

In [15]:
from sagemaker.xgboost import XGBoostProcessor

evaluation_processor = XGBoostProcessor(
    base_job_name="evaluation-processor",
    image_uri=config["image"],
    framework_version=config["framework_version"],
    instance_type=config["instance_type"],
    instance_count=1,
    role=role,
    sagemaker_session=config["session"],
)

INFO:sagemaker.image_uris:Ignoring unnecessary Python version: py3.
INFO:sagemaker.image_uris:Ignoring unnecessary instance type: ml.c5.xlarge.


In [16]:
from sagemaker.workflow.properties import PropertyFile

evaluation_report = PropertyFile(
    name="evaluation-report", output_name="evaluation", path="evaluation.json"
)

In [17]:
model_assets = train_model_step.properties.ModelArtifacts.S3ModelArtifacts

if USE_TUNING_STEP:
    model_assets = tune_model_step.get_top_model_s3_uri(
        top_k=0,
        s3_bucket=config["session"].default_bucket(),
    )

In [18]:
evaluate_model_step = ProcessingStep(
    name="evaluate-model",
    step_args=evaluation_processor.run(
        inputs=[
            ProcessingInput(
                source=split_and_transform_data_step.properties.ProcessingOutputConfig.Outputs[
                    "test"
                ].S3Output.S3Uri,
                destination="/opt/ml/processing/test",
            ),
            ProcessingInput(
                source=model_assets,
                destination="/opt/ml/processing/model",
            ),
        ],
        outputs=[
            ProcessingOutput(
                output_name="evaluation", source="/opt/ml/processing/evaluation"
            ),
        ],
        code=f"{CODE_FOLDER}/evaluation.py",
    ),
    property_files=[evaluation_report],
    cache_config=cache_config,
)

In [19]:
session3_pipeline = Pipeline(
    name="session3-pipeline",
    parameters=[dataset_location],
    steps=[
        split_and_transform_data_step,
        tune_model_step if USE_TUNING_STEP else train_model_step,
        evaluate_model_step,
    ],
    pipeline_definition_config=pipeline_definition_config,
    sagemaker_session=config["session"],
)

session3_pipeline.upsert(role_arn=role)
# session3_pipeline.start()

INFO:sagemaker.processing:Uploaded None to s3://football-data-kamil/session3-pipeline/code/79050343f4be619970fd5ef58ef667e5/sourcedir.tar.gz
INFO:sagemaker.processing:runproc.sh uploaded to s3://football-data-kamil/session3-pipeline/code/2c207c809cb0e0e9a1d77e5247f961f9/runproc.sh
INFO:sagemaker.processing:Uploaded None to s3://football-data-kamil/session3-pipeline/code/79050343f4be619970fd5ef58ef667e5/sourcedir.tar.gz
INFO:sagemaker.processing:runproc.sh uploaded to s3://football-data-kamil/session3-pipeline/code/2c207c809cb0e0e9a1d77e5247f961f9/runproc.sh


{'PipelineArn': 'arn:aws:sagemaker:eu-north-1:284415450706:pipeline/session3-pipeline',
 'ResponseMetadata': {'RequestId': '1c2d8aef-08a8-41bf-a49f-3830e69c03c2',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '1c2d8aef-08a8-41bf-a49f-3830e69c03c2',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '86',
   'date': 'Sat, 20 Apr 2024 13:35:44 GMT'},
  'RetryAttempts': 0}}

In [20]:
BASIC_MODEL_PACKAGE_GROUP = "basic-football"

In [21]:
from sagemaker.xgboost.model import XGBoostModel

xgb_model = XGBoostModel(
    model_data=model_assets,
    framework_version=config["framework_version"],
    sagemaker_session=config["session"],
    role=role,
)

In [22]:
from sagemaker.model_metrics import MetricsSource, ModelMetrics
from sagemaker.workflow.functions import Join

model_metrics = ModelMetrics(
    model_statistics=MetricsSource(
        s3_uri=Join(
            on="/",
            values=[
                evaluate_model_step.properties.ProcessingOutputConfig.Outputs[
                    "evaluation"
                ].S3Output.S3Uri,
                "evaluation.json",
            ],
        ),
        content_type="application/json",
    ),
)

In [23]:
from sagemaker.workflow.model_step import ModelStep


def create_registration_step(
        model,
        model_package_group_name,
        approval_status="Approved",
        content_types=["text/csv"],
        response_types=["application/json"],
        model_metrics=None,
        drift_check_baselines=None,
):
    """Create a Registration Step using the supplied parameters."""
    return ModelStep(
        name="register",
        step_args=model.register(
            model_package_group_name=model_package_group_name,
            approval_status=approval_status,
            model_metrics=model_metrics,
            drift_check_baselines=drift_check_baselines,
            content_types=content_types,
            response_types=response_types,
            inference_instances=[config["instance_type"]],
            transform_instances=[config["instance_type"]],
            framework_version=config["framework_version"],
            domain="MACHINE_LEARNING",
            task="CLASSIFICATION",
            framework="XGBOOST",
        ),
    )


register_model_step = create_registration_step(
    xgb_model,
    BASIC_MODEL_PACKAGE_GROUP,
    model_metrics=model_metrics,
)

INFO:sagemaker.image_uris:Ignoring unnecessary instance type: ml.c5.xlarge.


In [24]:
from sagemaker.workflow.functions import JsonGet
from sagemaker.workflow.fail_step import FailStep
from sagemaker.workflow.parameters import ParameterFloat
from sagemaker.workflow.condition_step import ConditionStep
from sagemaker.workflow.conditions import ConditionGreaterThanOrEqualTo

f1_threshold = ParameterFloat(name="f1_threshold", default_value=0.64)

fail_step = FailStep(
    name="fail",
    error_message=Join(
        on=" ",
        values=[
            "Execution failed because the model's f1 result was lower than",
            f1_threshold,
        ],
    ),
)

condition = ConditionGreaterThanOrEqualTo(
    left=JsonGet(
        step_name=evaluate_model_step.name,
        property_file=evaluation_report,
        json_path="metrics.f1.value",
    ),
    right=f1_threshold,
)

condition_step = ConditionStep(
    name="check-model-f1-result",
    conditions=[condition],
    if_steps=[register_model_step],
    else_steps=[fail_step],
)

In [25]:
session5_pipeline = Pipeline(
    name="session5-pipeline",
    parameters=[dataset_location, f1_threshold],
    steps=[
        split_and_transform_data_step,
        tune_model_step if USE_TUNING_STEP else train_model_step,
        evaluate_model_step,
        condition_step,
    ],
    pipeline_definition_config=pipeline_definition_config,
    sagemaker_session=config["session"],
)

session5_pipeline.upsert(role_arn=role)
# session5_pipeline.start()

INFO:sagemaker.processing:Uploaded None to s3://football-data-kamil/session5-pipeline/code/79050343f4be619970fd5ef58ef667e5/sourcedir.tar.gz
INFO:sagemaker.processing:runproc.sh uploaded to s3://football-data-kamil/session5-pipeline/code/2c207c809cb0e0e9a1d77e5247f961f9/runproc.sh
INFO:sagemaker.processing:Uploaded None to s3://football-data-kamil/session5-pipeline/code/79050343f4be619970fd5ef58ef667e5/sourcedir.tar.gz
INFO:sagemaker.processing:runproc.sh uploaded to s3://football-data-kamil/session5-pipeline/code/2c207c809cb0e0e9a1d77e5247f961f9/runproc.sh


_PipelineExecution(arn='arn:aws:sagemaker:eu-north-1:284415450706:pipeline/session5-pipeline/execution/4pgkqqnt8xee', sagemaker_session=<sagemaker.workflow.pipeline_context.PipelineSession object at 0x000002AF64D49AC0>)

In [26]:
response = sagemaker_client.list_model_packages(
    ModelPackageGroupName=BASIC_MODEL_PACKAGE_GROUP,
    ModelApprovalStatus="Approved",
    SortBy="CreationTime",
    MaxResults=1,
)

package = (
    response["ModelPackageSummaryList"][0]
    if response["ModelPackageSummaryList"]
    else None
)

# package

{'ModelPackageGroupName': 'basic-football',
 'ModelPackageVersion': 9,
 'ModelPackageArn': 'arn:aws:sagemaker:eu-north-1:284415450706:model-package/basic-football/9',
 'CreationTime': datetime.datetime(2024, 4, 20, 15, 43, 3, 15000, tzinfo=tzlocal()),
 'ModelPackageStatus': 'Completed',
 'ModelApprovalStatus': 'Approved'}

In [27]:
ENDPOINT = "football-endpoint"

In [32]:
from sagemaker import ModelPackage

if package:
    model_package = ModelPackage(
        model_package_arn=package['ModelPackageArn'],
        sagemaker_session=sagemaker_session,
        role=role,
    )

    model_package.deploy(
        endpoint_name=ENDPOINT,
        initial_instance_count=1,
        instance_type=config["instance_type"],
    )

INFO:sagemaker:Creating model with name: basic-football-2024-04-20-13-54-39-083
INFO:sagemaker:Creating endpoint-config with name football-endpoint
INFO:sagemaker:Creating endpoint with name football-endpoint


-----!

In [33]:
payload= '71,71,71,71,71,71,72,70,70,71,74,79,79,79,79,79,79,80,80,71,83,80,1.54,1.61,1.26,0.76,20,28,1.377149515625,2.5561203576634663,71.18,78.91,11.75,14.58,3.55,1.805,53.639,20.03,71.33,78.83,79.0,71.0,71.18181818181819,78.9090909090909,0,0,4.473,29.233606557377048,31.40637450199203,-2.172767944614982'

In [34]:
import numpy as np
import json
from sagemaker import Predictor

predictor = Predictor(endpoint_name=ENDPOINT)

try:
    response = predictor.predict(payload, initial_args={"ContentType": "text/csv"})
    response = json.loads(response.decode("utf-8"))

    print(json.dumps(response, indent=2))
    print(f"\nSpecies: {np.argmax(response['predictions'], axis=1)}")
except Exception as e:
    print(e)

0.430793434381485
'float' object is not subscriptable


In [44]:
response = sagemaker_client.describe_model_package(ModelPackageName=package['ModelPackageArn'])

print(response)

{'ModelPackageGroupName': 'basic-football', 'ModelPackageVersion': 7, 'ModelPackageArn': 'arn:aws:sagemaker:eu-north-1:284415450706:model-package/basic-football/7', 'CreationTime': datetime.datetime(2024, 4, 20, 13, 59, 22, 619000, tzinfo=tzlocal()), 'InferenceSpecification': {'Containers': [{'Image': '662702820516.dkr.ecr.eu-north-1.amazonaws.com/sagemaker-xgboost:1.7-1', 'ImageDigest': 'sha256:4366b7cee5fc070d6cf8b84a28304cf1baa359a6a2fbe8535d8c81961354326e', 'ModelDataUrl': 's3://football-data-kamil/sagemake-1gq9qqd5vo20-p5caqsJpFw-003-3e336a11/output/model.tar.gz', 'Environment': {'SAGEMAKER_CONTAINER_LOG_LEVEL': '20', 'SAGEMAKER_REGION': 'eu-north-1'}, 'Framework': 'XGBOOST', 'FrameworkVersion': '1.7-1'}], 'SupportedTransformInstanceTypes': ['ml.c5.xlarge'], 'SupportedRealtimeInferenceInstanceTypes': ['ml.c5.xlarge'], 'SupportedContentTypes': ['text/csv'], 'SupportedResponseMIMETypes': ['application/json']}, 'ModelPackageStatus': 'Completed', 'ModelPackageStatusDetails': {'Validat

In [ ]:
# Code to retrieve last model metric.
# from sagemaker.workflow.parameters import ParameterFloat
# import json
# 
# response = sagemaker_client.list_model_packages(ModelPackageGroupName=MODEL_PACKAGE_GROUP,
#                                                 SortBy='CreationTime',
#                                                 SortOrder='Descending',
#                                                 MaxResults=1)
# 
# if response['ModelPackageSummaryList']:
#     latest_model_package_arn = response['ModelPackageSummaryList'][0]['ModelPackageArn']
#     model_package_description = sagemaker_client.describe_model_package(ModelPackageName=latest_model_package_arn)
#     evaluation_metrics = model_package_description['ModelMetrics']['ModelQuality']['Statistics']
# 
#     s3_uri = evaluation_metrics['S3Uri']
#     s3_bucket, s3_key = s3_uri.replace("s3://", "").split("/", 1)
# 
#     obj = s3_client.get_object(Bucket=s3_bucket, Key=s3_key)
#     evaluation_json = json.loads(obj['Body'].read())
#     f1_metric = evaluation_json['metrics']['f1']['value']
# 
#     f1_threshold = ParameterFloat(name="f1_threshold", default_value=f1_metric)
#     print(f"F1 score from the latest model's evaluation: {f1_metric}")
# else:
#     print("No models found in the specified model package group.")
#     f1_metric = 0.67
#     f1_threshold = ParameterFloat(name="f1_threshold", default_value=f1_metric)

In [20]:
GROUND_TRUTH_LOCATION = f"{S3_LOCATION}/monitoring/groundtruth"
DATA_QUALITY_LOCATION = f"{S3_LOCATION}/monitoring/data-quality"
MODEL_QUALITY_LOCATION = f"{S3_LOCATION}/monitoring/model-quality"

In [21]:
from sagemaker.workflow.check_job_config import CheckJobConfig
from sagemaker.workflow.quality_check_step import DataQualityCheckConfig, QualityCheckStep
from sagemaker.model_monitor.dataset_format import DatasetFormat

data_quality_baseline_step = QualityCheckStep(
    name="generate-data-quality-baseline",
    check_job_config=CheckJobConfig(
        instance_type="ml.c5.xlarge",
        instance_count=1,
        volume_size_in_gb=20,
        sagemaker_session=config['session'],
        role=config['role'],
    ),

    quality_check_config=DataQualityCheckConfig(
        baseline_dataset=split_and_transform_data_step.properties.ProcessingOutputConfig.Outputs["train-baseline"].S3Output.S3Uri,
        dataset_format=DatasetFormat.csv(header=True, output_columns_position="END"),
        output_s3_uri=DATA_QUALITY_LOCATION
    ),
    skip_check=True,
    register_new_baseline=True,
    cache_config=cache_config
)

INFO:sagemaker.image_uris:Defaulting to the only supported framework/algorithm version: .
INFO:sagemaker.image_uris:Ignoring unnecessary instance type: None.


In [20]:
from sagemaker.workflow.functions import Join

transformation_pipeline_model = Join(
    on="/",
    values=[
        split_and_transform_data_step.properties.ProcessingOutputConfig.Outputs[
            "model"
        ].S3Output.S3Uri,
        "model.tar.gz",
    ],
)

In [21]:
from sagemaker.xgboost import XGBoostModel
from sagemaker.sklearn.model import SKLearnModel

preprocessing_model = SKLearnModel(
    model_data=transformation_pipeline_model,
    entry_point="preprocessing_component.py",
    source_dir=str(INFERENCE_CODE_FOLDER),
    framework_version="1.2-1",
    sagemaker_session=config["session"],
    role=config['role'],
)
xgb_model = XGBoostModel(
    model_data=tune_model_step.get_top_model_s3_uri(
        top_k=0, s3_bucket=config["session"].default_bucket()
    ) if USE_TUNING_STEP else train_model_step.properties.ModelArtifacts.S3ModelArtifacts,
    framework_version=config["framework_version"],
    sagemaker_session=config["session"],
    role=config["role"],
)

post_processing_model = SKLearnModel(
    model_data=transformation_pipeline_model,
    entry_point="postprocessing_component.py",
    source_dir=str(INFERENCE_CODE_FOLDER),
    framework_version="1.2-1",
    sagemaker_session=config["session"],
    role=config['role'],
)

from sagemaker.pipeline import PipelineModel

pipeline_model = PipelineModel(
    name="inference-model",
    models=[preprocessing_model, xgb_model, post_processing_model],
    sagemaker_session=config["session"],
    role=config['role'],
)

In [51]:
from sagemaker.workflow.steps import TransformStep
from sagemaker.workflow.model_step import ModelStep
from sagemaker.transformer import Transformer

create_model_step = ModelStep(
    name="create-model-step",
    display_name="create-model",
    step_args=pipeline_model.create(
        instance_type="ml.m5.xlarge"
    ),
)

transformer = Transformer(
    model_name=create_model_step.properties.ModelName,
    instance_type=config["instance_type"],
    instance_count=1,
    strategy="MultiRecord",
    accept="text/csv",
    assemble_with="Line",
    output_path=f"{S3_LOCATION}/transform",
    sagemaker_session=config["session"]
)

generate_test_predictions_step = TransformStep(
    name="generate-test-predictions",
    step_args=transformer.transform(
        # We will use the baseline set we generated when we split the data.
        # This set corresponds to the test split before the transformation step.
        data=split_and_transform_data_step.properties.ProcessingOutputConfig.Outputs["baseline"].S3Output.S3Uri,

        join_source="Input",
        split_type="Line",
        content_type="text/csv",

        # We want to output the first and the last field from the joint set.
        # The first field corresponds to the groundtruth, and the last field
        # corresponds to the prediction.
        output_filter="$[0,-1]",
    ),
    cache_config=cache_config
)

INFO:sagemaker.image_uris:Ignoring unnecessary instance type: ml.m5.xlarge.


ValueError: either step_args or transformer need to be given, but not both.

In [ ]:
from sagemaker.workflow.quality_check_step import ModelQualityCheckConfig

model_quality_baseline_step = QualityCheckStep(
    name="generate-model-quality-baseline",

    check_job_config=CheckJobConfig(
        instance_type="ml.c5.xlarge",
        instance_count=1,
        volume_size_in_gb=20,
        sagemaker_session=pipeline_session,
        role=execution_role,
    ),

    quality_check_config=ModelQualityCheckConfig(
        # We are going to use the output of the Transform Step to generate
        # the model quality baseline.
        baseline_dataset=generate_test_predictions_step.properties.TransformOutput.S3OutputPath,
        dataset_format=DatasetFormat.csv(header=False),

        # We need to specify the problem type and the fields where the prediction
        # and groundtruth are so the process knows how to interpret the results.
        problem_type="BinaryClassification",

        # Since the data doesn't have headers, SageMaker will autocreate headers for it.
        # _c0 corresponds to the first column, and _c1 corresponds to the second column.
        ground_truth_attribute="_c0",
        inference_attribute="_c1",

        output_s3_uri=MODEL_QUALITY_LOCATION,
    ),

    skip_check=True,
    register_new_baseline=True,
    cache_config=cache_config
)

In [ ]:
from sagemaker.model_metrics import MetricsSource, ModelMetrics
from sagemaker.drift_check_baselines import DriftCheckBaselines

model_metrics = ModelMetrics(
    model_data_statistics=MetricsSource(
        s3_uri=data_quality_baseline_step.properties.CalculatedBaselineStatistics,
        content_type="application/json",
    ),
    model_data_constraints=MetricsSource(
        s3_uri=data_quality_baseline_step.properties.CalculatedBaselineConstraints,
        content_type="application/json",
    ),
    model_statistics=MetricsSource(
        s3_uri=model_quality_baseline_step.properties.CalculatedBaselineStatistics,
        content_type="application/json",
    ),

    model_constraints=MetricsSource(
        s3_uri=model_quality_baseline_step.properties.CalculatedBaselineConstraints,
        content_type="application/json",
    ),
)

drift_check_baselines = DriftCheckBaselines(
    model_data_statistics=MetricsSource(
        s3_uri=data_quality_baseline_step.properties.BaselineUsedForDriftCheckStatistics,
        content_type="application/json",
    ),
    model_data_constraints=MetricsSource(
        s3_uri=data_quality_baseline_step.properties.BaselineUsedForDriftCheckConstraints,
        content_type="application/json",
    ),
    model_statistics=MetricsSource(
        s3_uri=model_quality_baseline_step.properties.BaselineUsedForDriftCheckStatistics,
        content_type="application/json",
    ),
    model_constraints=MetricsSource(
        s3_uri=model_quality_baseline_step.properties.BaselineUsedForDriftCheckConstraints,
        content_type="application/json",
    )
)

In [ ]:
PIPELINE_MODEL_PACKAGE_GROUP = "pipeline"

register_model_step = ModelStep(
    name="register",
    display_name="register-model",
    step_args=pipeline_model.register(
        model_package_group_name=PIPELINE_MODEL_PACKAGE_GROUP,
        model_metrics=model_metrics,
        approval_status="PendingManualApproval",
        content_types=["text/csv", "application/json"],
        response_types=["text/csv", "application/json"],
        inference_instances=[config["instance_type"]],
        transform_instances=[config["instance_type"]],
        domain="MACHINE_LEARNING",
        task="CLASSIFICATION",
        framework="XGBOOST",
        framework_version=config["framework_version"],
    ),
)

In [ ]:
from sagemaker.workflow.fail_step import FailStep
from sagemaker.workflow.functions import JsonGet
from sagemaker.workflow.functions import Join
from sagemaker.workflow.parameters import ParameterFloat
from sagemaker.workflow.conditions import ConditionGreaterThanOrEqualTo
from sagemaker.workflow.condition_step import ConditionStep

accuracy_threshold = ParameterFloat(name="f1_score_threshold", default_value=0.64)

condition_step = ConditionStep(
    name="check-model-f1_score",
    conditions=[
        ConditionGreaterThanOrEqualTo(
            left=JsonGet(
                step_name=evaluate_model_step.name,
                property_file=evaluation_report,
                json_path="metrics.f1.value",
            ),
            right=accuracy_threshold,
        )
    ],
    if_steps=[],

    else_steps=[
        FailStep(
            name="fail",
            error_message=Join(
                on=" ",
                values=[
                    "Execution failed because the model's accuracy was lower than",
                    accuracy_threshold,
                ],
            ),
        )
    ],
)

if not LOCAL_MODE:
    # SageMaker doesn't support running any of these steps in Local Mode.
    condition_step.if_steps.extend(
        [
            create_model_step,
            generate_test_predictions_step,
            model_quality_baseline_step,
            register_model_step,
        ]
    )

In [ ]:
from sagemaker.workflow.pipeline import Pipeline
from sagemaker.workflow.pipeline_definition_config import PipelineDefinitionConfig

pipeline = Pipeline(
    name="session4-pipeline",
    parameters=[
        dataset_location,
        accuracy_threshold,
    ],
    steps=[
        split_and_transform_data_step,
        tune_model_step if USE_TUNING_STEP else train_model_step,
        evaluate_model_step,
        condition_step
    ],
    pipeline_definition_config=PipelineDefinitionConfig(use_custom_job_prefix=True),
    sagemaker_session=config["session"],
)

if not LOCAL_MODE:
    # SageMaker doesn't support running any of these steps in Local Mode.
    pipeline.steps.extend([data_quality_baseline_step])

pipeline.upsert(role_arn=config["role"])

In [52]:
pipeline.start()

_PipelineExecution(arn='arn:aws:sagemaker:eu-north-1:284415450706:pipeline/session4-pipeline/execution/63u0ww1gmwlu', sagemaker_session=<sagemaker.workflow.pipeline_context.PipelineSession object at 0x000001CE6D363C80>)